<a href="https://colab.research.google.com/github/KordingLab/ENGR344/blob/master/tutorials/W3D1_What_should_we_do_when_data_has_problems/W3D1_Tutorial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

# Tutorial 3: Missing Data
**Week 3: What should we do when data has problems?**

**Content creators**: Rob Lindgren

**Content reviewers**: Konrad Kording, Keervani Kandala

**Content modifiers**: ---

**Modified Content reviewer**: ---


In [ ]:
# @title Due Dates Calendar

from ipywidgets import widgets
from IPython.display import display, IFrame, YouTubeVideo


out1 = widgets.Output()
with out1:
  calendar = IFrame(src="https://calendar.google.com/calendar/embed?src=356b9d2nspjttvgbb3tvgk2f58%40group.calendar.google.com&ctz=America%2FNew_York", width=600, height=480)
  display(calendar)

out = widgets.Tab([out1])
out.set_title(0, 'Calendar')

display(out)

___
# Tutorial Objectives

*Estimated timing of tutorial: __ minutes*

This is tutorial 3 in a 3-part series on how to handle data that has problems. In this tutorial, we will learn about missing data: what problems does it pose, how it is represented in base Python, NumPy, and Pandas, how to identify them, and how to remove or fill them. By the end of this tutorial, you will be able to:

- Explain the problems caused by missing values
- Explain the different types of missing values
- Identify missing values in NumPy and Pandas
- Remove missing values with NumPy and Pandas
- Fill in missing values with Pandas using a single value or a linear estimate

In [ ]:
# @title Tutorial slides
 
# @markdown These are the slides for the videos in all tutorials today
from IPython.display import IFrame

IFrame(src=f"https://mfr.ca-1.osf.io/render?url=https://osf.io/hncv7/?direct%26mode=render%26action=download%26mode=render", width=854, height=480)

---
# Setup

In [ ]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [ ]:
# @title Create different kinds of missing data

def mcar(df_in, var_list, true_probs):
  """Replaces values in the columns named by var_list with None such that
  the data in those columns are missing-completely-at-random.

  Args:
    df_in (DataFrame): Pandas DataFrame containing all variables in var_list.
    var_list (list of strings): List of variable names to which missing values
      will be assigned.
    true_prob (list of numbers): Each entry in true_prop indicates the probability
      of a missing value being assigned. It is the same length as var_list and
      the first entry corresponds to the first variable, the second to the second,
      etc. All entries must be between 0 and 1 inclusive.

  Returns:
    The DataFrame df_in, but with all variables in var_list now MCAR.
  """
  df = df_in.copy()

  n = df.shape[0]
  rng = np.random.RandomState(2)
  for var, t in zip(var_list, true_probs):
    f = 1-t
    mask = rng.choice([True, False], size=n, p=[t, f])
    df.loc[mask, [var]] = None

  return df

def mar(df_in, var_list, true_probs, fact_var, fact_val):
  """Replaces values in the columns named by var_list with None such that
  the data in those columns are missing-at-random.

  Args:
    df_in (DataFrame): Pandas DataFrame containing all variables in var_list.
    var_list (list of strings): List of variable names to which missing values
      will be assigned.
    true_prob (list of numbers): Each entry in true_prop indicates the probability
      of a missing value being assigned. It is the same length as var_list and
      the first entry corresponds to the first variable, the second to the second,
      etc. All entries must be between 0 and 1 inclusive.
    fact_var (string): Name of a factor variable in df_in that will determine 
      which observations are assignmed missing values and which are not.
    fact_val (dtype of df_in[fact_var]): The value of fact_var for which missing
      values will be assigned. 

  Returns:
    The DataFrame df_in, but with all variables in var_list now MCAR.
  """
  df = df_in.copy()
  df_sub = df[df[fact_var] == fact_val].copy()
  df = df[df[fact_var] != fact_val].copy()

  n = df_sub.shape[0]
  rng = np.random.RandomState(2)

  for var, t in zip(var_list, true_probs):
    f = 1-t
    mask = rng.choice([True, False], size=n, p=[t, f])
    df_sub.loc[mask, var] = None 

  df = df.append(df_sub)

  return df

def mnar(df_in):
  """Replaces values in the 'Highway mpg' column with None such that
  the data in those columns are missing-not-at-random.

  Args:
    df_in (DataFrame): Pandas DataFrame containing all variables in var_list.

  Returns:
    The DataFrame df_in, but with all 'Highway mpg' now MCAR.
  """
  df_bias = df_in.copy()
  df_bias = df.copy()
  #df_bias.loc[(df_bias['Highway mpg'] > 25) & (df_bias['Horsepower'] < 350), 'Highway mpg'] = None
  df_bias.loc[df_bias['Highway mpg'] > 25, 'Highway mpg'] = None

  return df_bias

def choose_missing(df_in):

  choice_dict = {'mcar' : mcar(df_in, var_list=['Highway mpg'], true_probs=[0.2]),
                 'mar' : mar(df_in, var_list=['Highway mpg'], true_probs=[0.2], fact_var='Driveline', fact_val='All-wheel drive'),
                 'mnar' : mnar(df_in)}

  rng = np.random.RandomState(1)
  fn_order = rng.choice(['mcar', 'mar', 'mnar'], size=3, replace=False)

  output_df = {'data' : {'A' : None, 'B' : None, 'C' : None},
               'name' : {'A' : None, 'B' : None, 'C' : None}}
  for fn,letter in zip(fn_order, ['A', 'B', 'C']):
    output_df['data'][letter] = choice_dict[fn]
    output_df['name'][letter] = fn

  return output_df

In [ ]:
# @title Plotting Functions

# Solution
def plt_cars(df):
  """Plot histograms of 'Horsepower' and ' Highway mpg' from the Cars dataset, 
  as well as a scatterplot of 'Horsepower' vs. 'Highway mpg'.

  Args:
    df (DataFrame): Cars dataset, with variables 'Horsepower' and 'Highway 'mpg'.

  Returns:
    None
  """

  # Compute means
  means = df.mean()

  # Create figure and axes objects
  fig_a, (ax1, ax2) = plt.subplots(1, 2)
  
  # Visualize 'Horsepower'
  ax1.hist('Horsepower', data=df)
  ax1.set_xlabel("Horsepower")
  ax1.set_ylabel("Number of vehicles")
  ax1.axvline(means['Horsepower'], color='Orange')

  # Visualize 'Highway mpg'
  ax2.hist('Highway mpg', data=df)
  ax2.set_xlabel("Highway mpg")
  ax2.set_ylabel("Number of vehicles")
  ax2.axvline(means['Highway mpg'], color='Orange')
  print(fig_a)

  print('\n')
  
  # Visualize the relationship between 'Horsepower' and 'Highway mpg'
  fig_b, ax = plt.subplots(1, 1)
  ax.scatter('Horsepower', 'Highway mpg', data=df)
  ax.set_xlabel('Horsepower')
  ax.set_ylabel('Highway mpg')
  print(fig_b)

def plt_reg(df_in, x, y, y_pred):
  """Creates a scatterplot of x and y from df_in, then overlays a lineplot
  of x and y_pred over top.

  Args:
    df_in (DataFrame): Dataset with variables labeled x and y.
    x (string): Name of variable to be plotted on x-axis.
    y (string): Name of variable to be plotted on y-axis.
    y_pred (array-like): Predicted y values resulting from regression.

  Returns:
    None
  """
  df = df_in.copy()
  fig, ax = plt.subplots(1, )
  ax.scatter(x, y, data=df)
  ax.plot(df[x], y_pred, color='red', linewidth=3)
  print(fig)


In [ ]:
# @title Helper function for linear regression
def regress(df_in, x_lab, y_lab):
  """Performs linear regression of y_lab on x_lab variables from 
  dataframe df_in. Returns results in a dictionary.

  Args:
    df_in (DataFrame): Pandas DataFrame with variables named
      x_lab and y_lab.
    x_lab (string): Name of independent variable for regression.
    y_lab (string): Name of dependent variable for regression.

  Returns:
    A dictionary of the following form...
      {'prediction' : predicted_values,
      'intercept' : intercept
      'coef' : coefficient}
  """
  
  # Takes a dataframe and two variable names from the dataframe
  # and returns a dictionary with the regression results.

  # Output dictionary looks like this...
  # {'prediction' : predicted_values,
  #  'intercept' : intercept
  #  'coef' : coefficient}
  df = df_in.copy()

  from sklearn.linear_model import LinearRegression

  x = df.loc[:, [x_lab]].values.reshape(-1, 1)  # values converts it into a numpy array
  y = df.loc[:, [y_lab]].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
  reg = LinearRegression()  # create object for the class
  reg.fit(x, y)  # perform linear regression
  y_pred = reg.predict(x)  # make predictions

  out = {'prediction' : reg.predict(x),
        'intercept' : reg.intercept_[0],
        'coef' : reg.coef_[0][0]}

  return out

In [ ]:
# @title Helper functions to detect and remove outliers
def detect_outliers(df_in, thresh):
  df = df_in.copy()
  df_z = (df - df.mean()) / df.std()
  df_out = np.abs(df_z) > thresh
  return df_out
  
def remove_outliers(df_in, thresh):
  df = df_in.copy()
  df_out = df[np.logical_not(detect_outliers(df, thresh)).all(axis=1)]
  return df_out

---
# Prepare Data

*Estimated timing to here from start of tutorial: 90 min*

In this tutorial we will learn how to identify and handline both outliers and missing values in our data while we continue to investigate the Cars dataset.

Once again, let's load the Cars dataset, subset it, and verify that we got the subset that we expected.


In [ ]:
data_url = 'https://raw.githubusercontent.com/RealTimeWeb/datasets/master/datasets/csv/cars/cars.csv'
df = pd.read_csv(data_url)[['ID', 'Driveline', 'Horsepower', 'Highway mpg']]
df.head()

Now let's use our `remove_outliers()` function from the last tutorial to remove all data points that are 3 or more standard deviations from the mean of their respective distributions.

In [ ]:
print(df.shape)
df = remove_outliers(df, 3)
print(df.shape)

# Section 1: What problems do missing values pose?

Missing values cause three key problems in data analysis:
- Reduction of statistical power
- Biased results
- Reduced representativeness of samples

In [ ]:
# @title Video 1: Missing data
from ipywidgets import widgets
from IPython.display import display, IFrame, YouTubeVideo


out1 = widgets.Output()
with out1:
  video = YouTubeVideo(id="-JwyylR2YEg", width=854, height=480, fs=1, rel=0)
  print(f'Video available at https://youtube.com/watch?v={video.id}')
  display(video)

out = widgets.Tab([out1])
out.set_title(0, 'Youtube')

display(out)

Discuss missing not at random (MNAR) as it relates to your field.

## Section 1.1: Reduction of statistical power

Statistical power is the likelihood of rejecting a null hypothesis when the null hypothesis is false, i.e. when you *should* reject the null hypothesis. Missing values, when not filled in somehow, reduce sample size which, in turn, reduces statistical power.

Even a small number of missing values per variable can be a serious issue when our analysis requires multiple variables. Take the following dataset, for example:

In [ ]:
bad_df = pd.DataFrame({'Speed' : [1, None, 3, 4, 5],
                       'Horsepower' : [6, 7, None, 9, 10],
                       'Mpg' : [11, 12, 13, None, 15]})
bad_df

Each column is missing just one value, but if we require all three for our analysis, then we are left with only two complete observations.

In [ ]:
bad_df.dropnav()

## Section 1.2: Bias and reduced representativeness of samples.

There may or may not be a pattern to the missingness of our data. The presence of a pattern will determine whether and how missing values result in a less representative sample and bias our results. Missing data can be:

- **Missing completely at random (MCAR):** Data is MCAR when the missingness of values is unrelated to what is being measured. Surveys may get lost in the mail, a faulty instrument may occassionally fail to register a measurement independently of what is being measured, and lab samples may be damaged before they can be tested. If data that is truly MCAR, the sample obtained will still be representative and estimates will be unbiased. 

- **Missing at random (MAR):** Data is MAR when the missingness of values is related variables in our dataset *other than* the variable with the missing value. If surveys are predictably lost in the mail in one zipcode but not others (and the zipcode is in our data), then the missing values are MAR. Similarly, if the faulty instrument more frequently fails on hot days rather than cold days (and we have a variable indicating hot and cold days in our data), those values are also MAR. As long as we properly account for the pattern with which our MAR data is missing, we can get a representative sample and an unbiased result.

- **Missing not at random (MNAR):** Data is MNAR when missingness is related to what is being measured. If our faulty instrument is a thermometer that regularly fails to record a temperature on hot days, those values would be MNAR. This is the most problematic type of missing data. The only way to get an unbiased result is to either go out and retrieve the missing data or accurately model it.


# Section 2: How are missing values represented?

## Section 2.1: In general

There are two ways to represent missing values:
- Masking
- Sentinel Values

Masking is typically performed using a boolean array with `True` entries indicating that value is missing and `False` entries indicating that it is not. The below mask tells us that the first and third entries are missing.

```
mask = [True, False, True, False, False]
```

Sentinel values are often data-specific, such as extreme values that would not be appropriate to the thing being measured. For example, the US Census counts the total number of people in a household. This number can never be negative, so -1 is a reasonable choice to represent missing values for this particular variable. 

```
hh_members = [2, 1, -1, 3, 5, -1, 2]
```

Sentinel values can also be conventional, such as the `NaN` (not-a-number) value used by NumPy. `NaN` is a special value in the IEEE floating-point specification set aside for undefined values.

In [ ]:
test = np.arange(10.0)
test[2] = np.NaN
test

There are trade-offs to each of these methods

- **Masking** requires the allocation of an additional array, which reduces storage and computational overhead. The degree to which this is a problem will depend on how large the allocated array needs to be.
- **Sentinel values in general** require extra logic for computations. For example, you can’t use a sum() function on a variable for which missing values are represented with a -1 without some extra steps.
- Using **extreme values** as sentinels reduces the range of values you can represent in your data.
- **Special values** like NaN are not available for all data types (there is no NaN for integers). Python libraries like NumPy and Pandas continue to update their handling of missing values to account for this, so it’s worth keeping an eye out for changes to missing value representation.


## Section 2.2: How NumPy and Pandas represent missing values

There are two representations of missing values in Python and its libraries NumPy and Pandas:
- `None`
- `NaN`

`None` in Python is of data type 'object', the most general data type. It can only be used in arrays that are also of type 'object'.

In [ ]:
arr = np.array([1, 2, None, 4])
arr

`None`: Base Python uses None to represent missing data
None is an object, so it can only be used in arrays with data type “object.” 
These arrays can hold a variety of data types including integers, floats, and strings, therefore they can’t really be optimized for performing calculations quickly. 
If you provide an array with a None object to a function like sum(), it will raise an error.

`NaN`: This is where NumPy comes in. NumPy relies on the IEEE floating-point NaN (not-a-number) np.nan
Because it is part of the IEEE 754 standard, it is recognized by most systems. 
This supports much faster operations.
This also means that aggregations are well-defined over NumPy arrays containing np.nan, so you can use np.sum() (which return np.nan if there is any np.nan in the input) or np.nansum() (which ignores the missing values). 


# Section 3: Identifying and missing values

## Section 3.1: Identifying missing values

NumPy and Pandas provide us with three handy methods for uncovering missing values: 
- `np.isnan()`
- `df.isnull()`
- `df.notnull()`

All three of them return a boolean array indicating either the presence (in the case of the first two) or absence (the last one) of missing values.

Returning to our NumPy array from earlier with the NaN at index 2...

In [ ]:
test = np.arange(10.0)
test[2] = np.NaN
test

We can identify the location of the missing value using `np.isnull()`.

In [ ]:
missings = np.isnan(test)
missings

And we can use this boolean array along with `np.logical_not()`, which applies the logical 'not' operator across elements of an array, to remove the missing value.

In [ ]:
test[np.logical_not(missings)]

We can use the `pd.isnull()` and `pd.notnull` operators on our Cars dataset. Notice that this is a method of the DataFrame object, so for the dataframe `df` you call it using `df.isnull()`.

In [ ]:
df.isnull()

In [ ]:
df.notnull()

### Coding Exercise 3.1: Count missing values by variable

Pandas DataFrames also have a `sum()` method that we can call by appending it to the DataFrame name, as in `df.sum()`. If you use the `sum()` method on a boolean array, Python will treat `True` values as 1 and `False` values as 0, allowing you to easily count the number of missing values in each column of a DateFrame.

*Exercise objective: print the number of missing values in each column of Cars dataset.*


In [ ]:
###########################################################################
## TODO for students: Use df.isnull() and df.sum() to count missing values.
raise NotImplementedError('student exercise: count missing values')
###########################################################################

print(...) # Hint: you can do this in one line by combining both methods

In [ ]:
# to_remove Solution

print(df.isnull().sum())


## Coding Exercise 3.2: Can you detect MCAR, MAR, and MNAR?

In the first cell below, we create three different DataFrames: `df_A`, `df_B`, and `df_C`.

- One of these DataFrames has 'Highway mpg' values missing-completely-at-random
- Another has 'Highway mpg' values missing-at-random with respect to the 'Driveline' variable, i.e., the missing values in 'Highway mpg' are somehow predictable from 'Driveline'
- And the third has 'Highway mpg' values missing-not-at-random, i.e., the missing values in 'Highway mpg' depend on real 'Highway mpg' and are not predictable from this dataset at all

Using the following tools, figure out which of the DataFrames below is MCAR, which is MAR, and which MNAR. 
- `plt_cars()`: Visualization is your friend!
- You can use a structure like `df.isnull().groupby(df['Driveline']).sum()` to calculate the number of the missing values for each value of Driveline.


In [ ]:
# @title Video 1: Detecting the nature of missing data
from ipywidgets import widgets
from IPython.display import display, IFrame, YouTubeVideo


out1 = widgets.Output()
with out1:
  video = YouTubeVideo(id="WM0WJen6QvM", width=854, height=480, fs=1, rel=0)
  print(f'Video available at https://youtube.com/watch?v={video.id}')
  display(video)

out = widgets.Tab([out1])
out.set_title(0, 'Youtube')

display(out)

In [ ]:
out_df = choose_missing(df)
df_A = out_df['data']['A']
df_B = out_df['data']['B']
df_C = out_df['data']['C']

In [ ]:
###########################################################################
## TODO for students: Identify the MCAR, MAR, and MNAR DataFrames.
raise NotImplementedError('Identify the MCAR, MAR, and MNAR DataFrames')
###########################################################################

In [ ]:
# to_remove Solution

# There is no obvious pattern to the distribution of missing values across 
# values of Driveline
print(df_A.isnull().groupby(df_A['Driveline']).sum())

# Both the histograms and scatterplot look as expected
plt_cars(df_A)

# Conclusion: df_A is likely MCAR


In [ ]:
# to_remove Solution contd.

# The missing values seem to be concentrated in Front-wheel drive
print(df_B.isnull().groupby(df_B['Driveline']).sum())

# The histogram for 'Highway mpg' is entirely cut off above 25
# This would mean the missingness of values depends on 'Highway mpg',
# i.e., all values greater than 25 are missing, so df_B must be MNAR
plt_cars(df_B)

# Conclusion: df_B is MNAR

In [ ]:
# to_remove Solution contd.

# This looks like good evidence that df_C is MAR: There is a relationship between Driveline and missing values
print(df_C.isnull().groupby(df_C['Driveline']).sum())

# The histograms and scatterplot look roughly as expected, so these don't tell us much
plt_cars(df_C)

# Conclusion: df_C is MAR

### Coding Exercise 3.2 Answer

Running the cell below will print out the answer to Coding Exercise 3.2.


In [ ]:
for key in out_df['name']:
  print('DataFrame df_' + key + ' is: ' + out_df['name'][key])

# Discussion
What can us data scientists do in the case of missing data? How should we think about them? Discuss where in the world data is missing and for what reasons.

# Section 4: Handling missing values

Pandas provides methods dropping rows with missing values, filling missing values with a constant, and interpolating missing values using one of several estimation methods.

- `df.dropna()`
- `df.fillna()`
- `df.interpolate()`

In [ ]:
# @title Video 1: Missing Intuitions
from ipywidgets import widgets
from IPython.display import display, IFrame, YouTubeVideo


out1 = widgets.Output()
with out1:
  video = YouTubeVideo(id="QGAbapaBl7I", width=854, height=480, fs=1, rel=0)
  print(f'Video available at https://youtube.com/watch?v={video.id}')
  display(video)

out = widgets.Tab([out1])
out.set_title(0, 'Youtube')

display(out)

### Coding Exercise 3.2: Estimate Missing Data

In this exercise we're going to compare different methods of filling missing data using `df.fillna()`.

- Fill in with the mean.
- Fill in with a linear estimate.

In [ ]:
###########################################################################
## TODO for students: Complete the following functions for filling in 
## missing data, one using the mean and one using a linear estimate.
raise NotImplementedError('student exercise: functions for filling in data')
###########################################################################

def fill_with_mean(df_in):
  df = df_in.copy()

  # Calculate the means of each column
  means = ...

  # Fill the missing values in df with the means calculated above
  # Hint: use inplace=True to modify the existing DataFrame without creating a 
  # new one
  ...
  return df

def fill_with_lin(df_in):
  df = df_in.copy()

  # Sort values by 'Horsepower'
  # Hint: use the sort_values() method for dataframes
  ... 

  # Interpolate using the linear method
  # Set the limit_direction parameter to 'both'
  ...
  return df

In [ ]:
# Solution
def fill_with_mean(df_in):
  df = df_in.copy()

  # Calculate the means of each column
  means = df.mean()

  # Fill the missing values in df with the means calculated above
  df.fillna(means, inplace=True) 

  return df.fillna(df.mean())

def fill_with_lin(df_in):
  df = df_in.copy()

  # Sort values by 'Horsepower'
  # Hint: use the sort_values() method for dataframes
  df.sort_values('Horsepower', inplace=True)

  # Interpolate using the linear method
  # Set the limit_direction parameter to 'both'
  df.interpolate(method='linear', inplace=True, limit_direction='both')
  
  return df

Now let's use our new functions and plot the results.

In [ ]:
# Fill with mean and plot
df_fill_m = fill_with_mean(df)
plt_cars(df_fill_m)

# Fill with linear estimate and plot
df_fill_lin = fill_with_lin(df)
plt_cars(df_fill_lin)


...And let's see how different methods of filling missing values affect our regression results.

In [ ]:
# Filling with the mean
reg_mean = regress(df_fill_m, 'Horsepower', 'Highway mpg') 
plt_reg(df_fill_m, 'Horsepower', 'Highway mpg', reg_mean['prediction'])
print('Intercept = ' + str(reg_mean['intercept']))
print('Coefficient = ' + str(reg_mean['coef']))

In [ ]:
# Filling with linear estimate
reg_lin = regress(df_fill_lin, 'Horsepower', 'Highway mpg') 
plt_reg(df_fill_lin, 'Horsepower', 'Highway mpg', reg_lin['prediction'])
print('Intercept = ' + str(reg_lin['intercept']))
print('Coefficient = ' + str(reg_lin['coef']))

# Section 4: Discussion
In this tutorial we have seen a broad range of problems and solutions for cases where we have missing data. Let us talk about the relevant issues. What can we handle as data scientists? What requires better data?

And how does all this relate to causality?


## Submit
Take the weekly survey/quiz and also submit your notebooks there. (Click the ?D logo below!)

<a href="https://airtable.com/shrJhXIeslRire1iD"><img src="https://github.com/KordingLab/ENGR344/blob/master/tutorials/static_344/SubmitButton.jpg?raw=1" alt="button link to survey" style="width:410px"></a>